In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1643785492760)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@54db1fa7


2부에서는 스파크의 구조적 API를 알아보았음<br/>
*대부분의 상황에서는 구조적 API를 사용하는 것이 좋음*<br/>
**그러나 비즈니스나 기술적 문제를 고수준 API를 사용해 모두 처리할 수 있는 것은 아님<br/>
이런 경우 스파크의 저수준 API를 사용해야 할 수도 있음**<br/>
스파크의 저수준 API는 RDD, SparkContext 그리고 accumulator와 broadcast variable과 같은 분산형 공유 변수(distributed shared variables) 등을 의미함<br/>
3부(저수준 API; 12~14장)에서는 저수준 API와 사용 방법에 대해 알아보겠음<br/>

# 12.1 저수준 API란

스파크에는 두 종류의 저수준 API가 있음<br/>
하나는 분산 데이터 처리를 위한 RDD이며, 다른 하나는 브로드캐스트 변수와 어큐뮬레이터와 같은 분산형 공유 변수를 배포하고 다루기 위한 API임<br/>

## 12.1.1 저수준 API는 언제 사용할까

다음과 같은 상황에서 저수준 API를 사용함<br/>
* 고수준 API에서 제공하지 않는 기능이 필요한 경우, 예를 들어 클러스터의 물리적 데이터의 배치를 아주 세밀하게 제어해야 하는 상황에서는 저수준 API가 필요함
* RDD를 사용해 개발된 기존 코드를 유지해야 하는 경우
* 사용자가 정의한 공유 변수를 다뤄야 하는 경우, 공유 변수는 14장에서 자세히 알아보겠음

위와 같은 상황에서만 저수준 API 기능을 사용해야 함<br/>
**그러나 스파크의 모든 워크로드는 저수준 기능을 사용하는 기초적인 형태로 컴파일되므로 이를 이해하는 것은 많은 도움이 될 수 있음**<br/>
*DataFrame 트랜스포메이션을 호출하면 실제로 다수의 RDD 트랜스포메이션으로 변환됨*<br/>
이러한 관계를 이해하면 점점 더 복잡해지는 워크로드를 디버깅하는 작업이 더욱 쉬워질 것임<br/>

스파크를 잘 알고 있는 숙련된 개발자라 하더라도 구조적 API 위주로 사용하는 것이 좋음<br/>
그러나 필요한 요건을 맞추기 위해 저수준 API를 사용해야 하는 경우가 발생할 수도 있음<br/>
즉, 이전 버전의 스파크에서 자체 구현한 파티셔너(partitioner)를 사용하거나 데이터 파이프라인이 실행되는 동안 변수값을 갱신하고 추적하는 데 저수준 API가 필요할 수 있음<br/>
저수준 API는 세밀한 제어 방법을 제공하여 개발자가 치명적인 실수를 하지 않도록 도와주기도 함<br/>

## 12.1.2 저수준 API는 어떻게 사용할까

SparkContext는 저수준 API 기능을 사용하기 위한 진입 지점임<br/>
스파크 클러스터에서 연산을 수행하는 데 필요한 도구인 SparkSession을 이용해 SparkContext에 접근할 수 있음<br/>
SparkContext는 15장에서 더 자세히 알아보겠음<br/>
지금은 다음 명령을 사용해 SparkContext에 접근할 수 있다는 사실만 알고 있으면 됨<br/>

In [2]:
spark.sparkContext

res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4b9afd8b


# 12.2 RDD 개요

RDD는 스파크 1.x 버전의 핵심 API임<br/>
스파크 2.x 버전에서도 사용할 수 있지만 잘 사용하지 않음<br/>
그러나 앞서 언급했듯이 **사용자가 실행한 모든 DataFrame이나 Dataset 코드는 RDD로 컴파일됨**<br/>
또한 18장에서 알아볼 스파크 UI에서 RDD 단위로 잡이 수행됨을 알 수 있음<br/>
그러므로 적어도 RDD가 무엇인지, 어떻게 사용하는지 기본적으로 이해하고 있어야 함<br/>

*간단히 말해 RDD는 불변성을 가지며 병렬로 처리할 수 있는 파티셔닝된 레코드의 모음*임<br/>
**DataFrame의 각 레코드는 스키마를 알고 있는 필드로 구성된 구조화된 로우인 반면, RDD의 레코드는 그저 프로그래머가 선택하는 자바, 스칼라, 파이썬의 객체일 뿐**임<br/>

RDD의 모든 레코드는 자바나 파이썬의 객체이므로 완벽하게 제어할 수 있음<br/>
이러한 객체에는 사용자가 원하는 포맷을 사용해 원하는 모든 데이터를 저장할 수 있음<br/>
*개발자는 강력한 제어권을 가질 수 있지만, 잠재적인 문제가 발생할 수 있음<br/>
모든 값을 다루거나, 값 사이의 상호작용 과정을 반드시 수동으로 정의해야 함*<br/>
즉, 어떤 처리를 하더라도 '바퀴를 다시 발명'해야 함<br/>
또한 *구조적 API와는 다르게 레코드의 내부 구조를 스파크에서 파악할 수 없으므로 최적화를 하려면 훨씬 많은 수작업이 필요*함<br/>
예를 들어 스파크의 구조적 API는 자동으로 데이터를 최적화하고 압축된 바이너리 포맷으로 저장함<br/>
반면 저수준 API에서 동일한 공간 효율성과 성능을 얻으려면 객체에 이런 포맷 타입을 구현해 모든 저수준 연산 과정에서 사용해야 함<br/>
이와 유사하게 스파크 SQL에서 자동으로 수행되는 필터 재정렬과 집계 같은 최적화 기법도 직접 구현해야 함<br/>
**그러므로 스파크의 구조적 API를 사용할 것을 강력하게 권고함**<br/>

RDD API는 11장에서 알아본 Dataset과 유사하지만 RDD는 구조화된 데이터 엔진을 사용해 데이터를 저장하거나 다루지 않음<br/>
하지만 RDD와 Dataset 사이의 전환은 매우 쉬우므로 두 API를 사용해 각 API의 장점을 동시에 활용할 수 있음<br/>

## 12.2.1 RDD 유형

스파크 API 문서를 살펴보면 RDD에 수많은 하위 클래스가 존재한다는 사실을 알 수 있음<br/>
**RDD는 DataFrame API에서 최적화된 물리적 실행 계획을 만드는 데 대부분 사용됨**<br/>

사용자는 두 가지 타입의 RDD를 만들 수 있음<br/>
* 제네릭 RDD 타입
* 키 기반의 집계가 가능한 키-값 RDD

목적에 맞게 두 RDD 중 하나를 선택할 수 있음<br/>
**둘 다 객체의 컬렉션을 표현하지만 키-값 RDD는 특수 연산뿐만 아니라 키를 이용한 사용자 지정 파티셔닝 개념을 가지고 있음**<br/>

RDD를 명확히 정의해보겠음<br/>
내부적으로 각 RDD는 다음 5가지 주요 속성으로 구분됨<br/>
* 파티션의 목록
* 각 조각을 연산하는 함수
* 다른 RDD와의 의존성 목록
* 부가적으로 키-값 RDD를 위한 Partitioner(예: RDD는 해시 파티셔닝되어 있다고 말할 수 있음)
* 부가적으로 각 조각을 연산하기 위한 기본 위치 목록(예: HDFS 파일의 블록 위치)

*NOTE*<br/>
*Partitioner는 RDD를 사용하는 주된 이유 중 하나일 것임<br/>
올바른 사용자 정의 Partitioner를 사용한다면 성능과 안정성이 크게 향상될 수 있음*<br/>
이와 관련된 내용은 13장에서 키-값 쌍 RDD를 소개할 때 더 자세히 알아보겠음<br/>

이러한 속성은 사용자 프로그램을 스케줄링하고 실행하는 스파크의 모든 처리 방식을 결정함<br/>
각 RDD 유형은 앞서 나열한 각 속성에 대한 구현체를 가지고 있음<br/>
사용자는 각 속성을 구현하여 새로운 데이터 소스를 정의할 수도 있음<br/>

RDD는 앞서 소개한 스파크 프로그래밍 패러다임을 그대로 따름<br/>
RDD 역시 분산 환경에서 데이터를 다루는 데 필요한 지연 처리 방식의 transformation과 즉시 실행 방식의 action을 제공함<br/>
그리고 DataFrame과 Dataset의 트랜스포메이션, 액션과 동일한 방식으로 동작함<br/>
**하지만 RDD에는 '로우'라는 개념이 없음**<br/>
개별 레코드는 자바, 스칼라, 파이썬 객체일 뿐이며 구조적 API에서 제공하는 여러 함수를 사용하지 못하기 때문에 수동으로 처리해야 함<br/>

RDD API는 스칼라와 자바뿐만 아니라 파이썬에서도 사용할 수 있음<br/>
스칼라와 자바를 사용하는 경우에는 대부분 비슷한 성능이 나오지만, 원형 객체를 다룰 때는 큰 성능 손실이 발생할 수 있음<br/>
반면 파이썬을 사용해 RDD를 다룰 때는 상당한 성능 저하가 발생할 수 있음<br/>
**파이썬으로 RDD를 실행하는 것은 파이썬으로 만들어진 사용자 정의 함수를 사용해 로우마다 적용하는 것과 동일하다고 볼 수 있음<br/>
6장에서 본 것처럼 직렬화 과정을 거친 데이터를 파이썬 프로세스에 전달하고, 파이썬에서 처리가 끝나면 다시 직렬화하여 JVM에 반환함<br/>
그러므로 파이썬을 사용해 RDD를 다룰 때는 높은 오버헤드가 발생함**<br/>
과거에는 많은 사람이 이러한 파이썬 코드를 운영 환경에서 많이 사용했음<br/>
그러나 꼭 필요한 경우가 아니라면 파이썬에서도 구조적 API를 사용하는 것이 좋음<br/>

*의문사항*<br/>
위에서 파이썬을 사용해 RDD를 다룰 때 발생하는 높은 오버헤드에 대해 설명했는데, 해당 오버헤드가 파이썬에서 구조적 API를 사용하는 경우 발생하지 않나?<br/>

## 12.2.2 RDD는 언제 사용할까

정말 필요한 경우가 아니라면 수동으로 RDD를 생성하면 안 됨<br/>
DataFrame은 RDD보다 더 효율적이고 안정적이며 표현력이 좋음<br/>

물리적으로 분산된 데이터(자체적으로 구성한 데이터 파티셔닝)에 세부적인 제어가 필요할 때 RDD를 사용하는 것이 가장 적합함<br/>

## 12.2.3 Dataset과 RDD의 케이스 클래스

Dataset과 케이스를 사용해서 만들어진 RDD의 차이점은 무엇일까?<br/>
Dataset은 구조적 API가 제공하는 풍부한 기능과 최적화 기법을 제공한다는 것이 가장 큰 차이점임<br/>
Dataset을 사용하면 JVM 데이터 타입과 스파크 데이터 타입 중 어떤 것을 쓸지 고민하지 않아도 됨<br/>
어떤 것을 사용하더라도 성능은 동일하므로 가장 쉽게 사용할 수 있고 유연하게 대응할 수 있는 데이터 타입을 선택하면 됨<br/>

# 12.3 RDD 생성하기

지금까지 RDD의 주요 속성을 알아보았음<br/>
이제 RDD를 사용하는 방법을 알아보겠음<br/>

## 12.3.1 DataFrame, Dataset으로 RDD 생성하기

RDD를 얻을 수 있는 가장 쉬운 방법은 기존에 사용하던 DataFrame이나 Dataset을 이용하는 것임<br/>
기존에 사용하던 DataFrame이나 Dataset의 rdd 메서드를 호출하면 쉽게 RDD로 변환할 수 있음<br/>
Dataset\[T\]를 RDD로 변환하면 데이터 타입 T를 가진 RDD를 얻을 수 있음<br/>
이러한 처리 방식은 자바와 스칼라에서만 사용할 수 있음<br/>

In [3]:
// 스칼라 코드: Dataset[Long]을 RDD[Long]으로 변환
spark.range(500).rdd

res2: org.apache.spark.rdd.RDD[Long] = MapPartitionsRDD[5] at rdd at <console>:27


파이썬에는 DataFrame만 존재하며 Dataset을 사용할 수 없으므로 Row 타입의 RDD를 얻게 됨<br/>

In [4]:
spark.range(10).toDF().rdd.map(rowObject => rowObject.getLong(0))

res3: org.apache.spark.rdd.RDD[Long] = MapPartitionsRDD[12] at map at <console>:26


RDD를 사용해 DataFrame이나 Dataset을 생성할 때도 동일한 방법을 사용함<br/>
RDD의 toDF 메서드를 호출하기만 하면 됨<br/>

In [5]:
spark.range(10).rdd.toDF()

res4: org.apache.spark.sql.DataFrame = [value: bigint]


rdd 메서드는 Row 타입을 가진 RDD를 생성함<br/>
Row 타입은 스파크가 구조적 API에서 데이터를 표현하는 데 사용하는 내부 카탈리스트 포맷임<br/>
이 기능을 사용하면 상황에 따라 구조적 API와 저수준 API를 오고가게 만들 수 있음<br/>

RDD API와 11장에서 알아본 Dataset API는 구조적 API가 제공하는 여러 가지 편리한 기능과 인터페이스를 가지고 있지 않으므로 매우 유사하게 느껴질 수 있음<br/>

## 12.3.2 로컬 컬렉션으로 RDD 생성하기

컬렉션 객체를 RDD로 만들려면 (SparkSession 안에 있는) sparkContext의 parallelize 메서드를 호출해야 함<br/>
이 메서드는 단일 노드에 있는 컬렉션을 병렬 컬렉션으로 전환함<br/>
또한 파티션 수를 명시적으로 지정할 수 있음<br/>
다음 예제에서는 두 개의 파티션을 가진 병렬 컬렉션 객체를 만듦<br/>

In [6]:
val myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
val words = spark.sparkContext.parallelize(myCollection, 2)

myCollection: Array[String] = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple)
words: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[19] at parallelize at <console>:25


RDD에 이름을 지정하면 스파크 UI에 지정한 이름으로 RDD가 표시됨<br/>

In [7]:
words.setName("myWords")
words.name

res5: String = myWords


## 12.3.3 데이터소스로 RDD 생성하기

데이터 소스나 텍스트 파일을 이용해 RDD를 직접 생성할 수도 있지만 DataSource API를 사용하는 것이 더 바람직함<br/>
RDD에는 DataFrame이 제공하는 'DataSource API'라는 개념이 없음<br/>
*RDD는 주로 RDD 간의 의존성 구조와 파티션 목록을 정의함*<br/>
DataSource API는 데이터를 읽는 가장 좋은 방법임<br/>
또한 sparkContext를 사용해 데이터를 RDD로 읽을 수 있음<br/>
다음은 줄 단위로 텍스트 파일을 읽는 예제임<br/>

In [8]:
/*
spark.sparkContext.textFile("/some/path/withTextFiles")
*/

코드 예제는 여러 텍스트 파일의 각 줄을 레코드로 가진 RDD를 생성함<br/>
또한 텍스트 파일 하나를 레코드로 읽어야 하는 경우도 있음<br/>
예를 들어 커다란 JSON 객체나 어떤 문서로 구성된 파일을 개별 레코드로 처리해야 할 수도 있음<br/>

In [9]:
/*
spark.sparkContext.wholeTextFiles("/some/path/withTextFiles")
*/

생성된 RDD에서 파일명은 첫 번째 객체인 RDD의 키가 되며, 텍스트 파일의 값은 두 번째 문자열 객체인 RDD의 값이 됨<br/>

# 12.4 RDD 다루기

RDD를 다루는 방식은 DataFrame을 다루는 방식과 매우 유사함<br/>
이전에도 언급했지만 RDD는 스파크 데이터 타입 대신 자바나 스칼라의 객체를 다룬다는 사실이 가장 큰 차이점임<br/>
또한 *연산을 단순화하는 '헬퍼' 메서드나 함수도 DataFrame에 비해 많이 부족함<br/>
그러므로 필터, 맵 함수, 집계 그리고 DataFrame의 다양한 함수를 사용자가 직접 정의해야 함*<br/>

RDD에서 데이터를 다루는 방법을 알아보기 전에 이전 예제에서 만들었던 RDD(words)에 다양한 기능을 정의해보겠음<br/>

# 12.5 트랜스포메이션

대부분의 RDD 트랜스포메이션은 구조적 API에서 사용할 수 있는 기능을 가지고 있음<br/>
DataFrame이나 Dataset과 동일하게 RDD에 **트랜스포메이션**을 지정해 새로운 RDD를 생성할 수 있음<br/>
이때 RDD에 포함된 데이터를 다루는 함수에 따라 다른 RDD에 대한 의존성도 함께 정의함<br/>

## 12.5.1 distinct

RDD의 distinct 메서드를 호출하면 RDD에서 중복된 데이터를 제거함<br/>

In [10]:
words.distinct().count()

res8: Long = 10


## 12.5.2 filter

필터링은 SQL의 where 조건절을 생성하는 것과 비슷함<br/>
RDD의 레코드를 모두 확인하고 조건 함수를 만족하는 레코드만 반환함<br/>
**조건 함수는 필터 함수로 동작하므로 불리언 타입을 반환해야 함**<br/>
모든 로우는 어떤 경우라도 입력값을 가지고 있어야 함<br/>
다음 예제는 문자 'S'로 시작하는 단어만 남도록 RDD를 필터링함<br/>

In [11]:
def startsWithS(individual:String) = {
    individual.startsWith("S")
}

startsWithS: (individual: String)Boolean


조건 함수를 정의했으니 데이터를 필터링할 차례임<br/>
11장을 읽어본 독자라면 RDD 레코드별로 처리하는 함수를 사용해봤으므로 친숙하게 느껴질 것<br/>
이 함수 역시 RDD의 각 레코드를 개별적으로 처리함<br/>

In [12]:
words.filter(word => startsWithS(word)).collect()

res9: Array[String] = Array(Spark, Simple)


이 코드는 Spark와 Simple이라는 결과를 사용한 언어의 데이터 타입으로(Dataset API처럼) 반환함<br/>
그 이유는 데이터를 Row 타입으로 강제 변환하거나 데이터를 수집한 다음 변환할 필요가 없기 때문<br/>

## 12.5.3 map

map 메서드는 11장에서 보았던 것과 동일한 작업을 함<br/>
주어진 입력을 원하는 값으로 반환하는 함수를 명시하고 레코드별로 적용함<br/>
설명을 위해 이전 예제와 유사한 처리를 수행해보겠음<br/>
예제에서는 현재 단어를 '단어', '단어의 시작 문자', '첫 문자가 S인지 아닌지' 순서로 매핑함<br/>

In [13]:
val words2 = words.map(word => (word, word(0), word.startsWith("S")))

words2: org.apache.spark.rdd.RDD[(String, Char, Boolean)] = MapPartitionsRDD[24] at map at <console>:26


In [14]:
words2.foreach(println)

(Spark,S,true)
(The,T,false)
(Definitive,D,false)
(Guide,G,false)
(:,:,false)
(Big,B,false)
(Data,D,false)
(Processing,P,false)
(Made,M,false)
(Simple,S,true)


새로 만든 함수의 3번째 반환값인 불리언값으로 필터링할 수 있음<br/>

In [15]:
words2.filter(record => record._3).take(5)

res11: Array[(String, Char, Boolean)] = Array((Spark,S,true), (Simple,S,true))


### flatmap

flatMap 메서드는 map 함수의 확장 버전임<br/>
때로는 단일 로우를 여러 로우로 변환해야 하는 경우가 있음<br/>
예를 들어 flatMap 메서드를 사용해 단어(word)를 문자(character) 집합으로 변환할 수 있음<br/>
각 단어는 여러 문자로 구성되어 있으므로 flatMap 메서드를 사용해 다수의 로우로 변환할 수 있음<br/>
flatMap은 확장 가능한 map 함수의 출력을 반복 처리할 수 있는 형태로 반환함<br/>

In [16]:
words.flatMap(word => word.toSeq).take(5)

res12: Array[Char] = Array(S, p, a, r, k)


## 12.5.4 sortBy

RDD를 정렬하려면 sortBy 메서드를 사용함<br/>
다른 RDD 작업과 마찬가지로 함수를 지정해 RDD의 데이터 객체에서 값을 추출한 다음 값을 기준으로 정렬함<br/>
예를 들어 다음 예제는 단어 길이가 가장 긴 것부터 짧은 순으로 정렬함<br/>

In [17]:
words.sortBy(word => word.length() * -1).take(2)

res13: Array[String] = Array(Definitive, Processing)


## 12.5.5 randomSplit

randomSplit 메서드는 RDD를 임의로 분할해 RDD 배열을 만들 때 사용하며, 가중치와 난수 시드(random seed)로 구성된 배열을 파라미터로 사용함<br/>

In [18]:
val fiftyFiftySplit = words.randomSplit(Array[Double](0.5, 0.5))

fiftyFiftySplit: Array[org.apache.spark.rdd.RDD[String]] = Array(MapPartitionsRDD[32] at randomSplit at <console>:26, MapPartitionsRDD[33] at randomSplit at <console>:26)


위 코드는 개별로 다룰 수 있는 RDD 배열을 결과로 반환함<br/>

In [19]:
fiftyFiftySplit.foreach(println)

MapPartitionsRDD[32] at randomSplit at <console>:26
MapPartitionsRDD[33] at randomSplit at <console>:26


# 12.6 액션

DataFrame과 Dataset에서 했던 것처럼 지정된 트랜스포메이션 연산을 시작하려면 액션을 사용함<br/>
액션은 데이터를 드라이버로 모으거나 외부 데이터소스로 내보낼 수 있음<br/>

## 12.6.1 reduce

RDD의 모든 값을 하나의 값으로 만들려면 reduce 메서드를 사용함<br/>
예를 들어 정수형 집합이 주어졌다면 두 개의 입력값을 하나로 줄이는 함수를 사용해 합계를 구할 수 있음<br/>
함수형 프로그래밍에 경험이 있다면 익숙한 개념일 것임<br/>

In [20]:
spark.sparkContext.parallelize(1 to 20).reduce(_ + _) // 값은 210

res15: Int = 210


단어 집합에서 가장 긴 단어를 찾는 예제는 reduce 메서드를 사용해 처리할 수 있음<br/>
올바른 함수를 정의하는 것이 다음 예제의 핵심임<br/>

In [21]:
def wordLengthReducer(leftWord:String, rightWord:String): String = {
    if(leftWord.length > rightWord.length)
        return leftWord
    else
        return rightWord
}

wordLengthReducer: (leftWord: String, rightWord: String)String


In [22]:
words.reduce(wordLengthReducer)

res16: String = Processing


In [23]:
words.reduce(wordLengthReducer)

res17: String = Definitive


wordLengthReducer 함수는 두 개의 입력값을 하나의 결과로 만들기 때문에 reduce 메서드를 설명하는 데 적합함<br/>
**파티션에 대한 리듀스 연산은 비결정적 특성을 가짐<br/>
따라서 단어의 길이가 10인 'definitive'나 'processing' 중 하나가 leftWord 변수에 할당될 수 있음<br/>
즉, reduce 메서드를 실행할 때마다 다른 결과를 반환할 수 있음**<br/>

## 12.6.2 count

count 함수를 사용하면 RDD의 전체 로우 수를 알 수 있음<br/>

In [24]:
words.count()

res18: Long = 10


### countApprox

이 함수의 반환 결과는 조금 이상해보일 수 있지만 꽤 정교한 편임<br/>
이 함수는 앞서 알아본 count 함수의 근사치를 제한 시간 내에 계산함<br/>
제한 시간을 초과하면 불완전한 결과를 반환할 수도 있음<br/>

신뢰도(confidence)는 실제로 연산한 결과와의 오차율을 의미함<br/>
즉, countApprox 메서드의 신뢰도를 0.9로 설정하고 반복적으로 호출하면 실제 연산 결과와 동일한 값이 90% 이상 포함될 것으로 기대할 수 있음<br/>
신뢰도는 \[0, 1\] 범위의 값이어야 하며, 범위를 벗어나면 예외가 발생함<br/> 

In [25]:
val confidence = 0.95
val timeoutMilliseconds = 400
words.countApprox(timeoutMilliseconds, confidence)

confidence: Double = 0.95
timeoutMilliseconds: Int = 400
res19: org.apache.spark.partial.PartialResult[org.apache.spark.partial.BoundedDouble] = (final: [10.000, 10.000])


### countApproxDistinct

countApproxDistinct 메서드에는 streamlib 관련 논문인 \<HyperLogLog in Practice: Algorithmic Engineering of a State-of-the-Art Cardinality Estimation Algorithm\>을 기반으로 한 2가지 구현체가 있음<br/>

countApproxDistinct 메서드의 첫 번째 구현체에서는 상대 정확도(relative accuracy)를 파라미터로 사용함<br/>
이 값을 작게 설정하면 더 많은 메모리 공간을 사용하는 카운터가 생성됨<br/>
설정값은 반드시 0.000017보다 커야 함<br/>

In [26]:
words.countApproxDistinct(0.05)

res20: Long = 10


다른 구현체를 사용하면 동작을 세부적으로 제어할 수 있음<br/>
상대 정확도를 지정할 때 두 개의 파라미터를 사용함<br/>
하나는 '일반(regular)' 데이터를 위한 파라미터이며, 다른 하나는 희소 표현을 위한 파라미터임<br/>

두 인수 p와 sp는 정밀도와 희소 정밀도를 의미함<br/>
상대 정확도는 대략 1.054/sqrt(2^P)임<br/>
카디널리티가 작을 때 0이 아닌 값(sp > p)을 설정하면 메모리 소비를 줄이면서 정확도를 증가시킬 수 있음<br/>
두 파라미터 모두 정수 데이터 타입을 사용함<br/>

In [27]:
words.countApproxDistinct(4, 10)

res21: Long = 10


### countByValue

이 메서드는 RDD 값의 개수를 구함<br/>
하지만 결과 데이터셋을 드라이버의 메모리로 읽어들여 처리함<br/>
이 메서드를 사용하면 executor의 연산 결과가 드라이버 메모리에 모두 적재됨<br/>
따라서 결과가 작은 경우에만 사용해야 함<br/>
그러므로 이 메서드는 전체 로우 수나 고유 아이템 수가 작은 경우에만 사용하는 것이 좋음<br/>

In [28]:
words.countByValue()

res22: scala.collection.Map[String,Long] = Map(Definitive -> 1, Simple -> 1, Processing -> 1, The -> 1, Spark -> 1, Made -> 1, Guide -> 1, Big -> 1, : -> 1, Data -> 1)


### countByValueApprox

count 함수와 동일한 연산을 수행하지만 근사치를 계산함<br/>
이 함수는 반드시 지정된 제한시간(첫 번째 파라미터) 내에 처리해야 함<br/>
제한 시간을 초과하면 불완전한 결과를 반환할 수 있음<br/>

신뢰도는 실제로 연산한 결과와의 오차율을 의미함<br/>
즉, countByValueApprox 메서드의 신뢰도를 0.9로 설정하고 반복적으로 호출하면 실제 연산 결과와 동일한 값이 90% 이상 포함될 것으로 기대할 수 있음<br/>
신뢰도는 \[0,1\] 범위의 값이어야 하며, 범위를 벗어나면 예외가 발생함<br/> 

In [29]:
words.countByValueApprox(1000, 0.95)

res23: org.apache.spark.partial.PartialResult[scala.collection.Map[String,org.apache.spark.partial.BoundedDouble]] = (final: Map(Definitive -> [1.000, 1.000], Simple -> [1.000, 1.000], Processing -> [1.000, 1.000], The -> [1.000, 1.000], Spark -> [1.000, 1.000], Made -> [1.000, 1.000], Guide -> [1.000, 1.000], Big -> [1.000, 1.000], : -> [1.000, 1.000], Data -> [1.000, 1.000]))


## 12.6.3 first

first 메서드는 데이터셋의 첫 번째 값을 반환함<br/>

In [30]:
words.first()

res24: String = Spark


## 12.6.4 max와 min

max와 min 메서드는 각각 최댓값과 최솟값을 반환함<br/>

In [32]:
spark.sparkContext.parallelize(1 to 20).max()

res26: Int = 20


In [33]:
spark.sparkContext.parallelize(1 to 20).min()

res27: Int = 1


## 12.6.5 take

take와 이것의 파생 메서드는 RDD에서 가져올 값의 개수를 파라미터로 사용함<br/>
**이 메서드는 먼저 하나의 파티션을 스캔함<br/>
그 다음에 해당 파티션의 결과 수를 이용해 파라미터로 지정된 값을 만족하는 데 필요한 추가 파티션 수를 예측함**<br/>

또한 takeOrdered, takeSample 그리고 top과 같은 다양한 유사 함수가 존재함<br/>
RDD에서 고정 크기의 임의 표본 데이터를 얻기 위해 takeSample 함수를 사용할 수 있음<br/>
takeSample 함수는 withReplacement, 임의 표본 수, 난수 시드값을 파라미터로 사용함<br/>
top 함수는 암시적(implicit) 순서에 따라 최상위값을 선택한다는 점에서 takeOrdered 함수와는 반대되는 개념으로 볼 수 있음<br/>

In [34]:
words.take(5)

res28: Array[String] = Array(Spark, The, Definitive, Guide, :)


In [35]:
words.takeOrdered(5)

res29: Array[String] = Array(:, Big, Data, Definitive, Guide)


In [36]:
words.top(5)

res30: Array[String] = Array(The, Spark, Simple, Processing, Made)


In [37]:
val withReplacement = true
val numberToTake = 6
val randomSeed = 100L

words.takeSample(withReplacement, numberToTake, randomSeed)

withReplacement: Boolean = true
numberToTake: Int = 6
randomSeed: Long = 100
res31: Array[String] = Array(Guide, Spark, :, Simple, Simple, Spark)


# 12.7 파일 저장하기

파일 저장은 데이터 처리 결과를 일반 텍스트 파일로 쓰는 것을 의미함<br/>
RDD를 사용하면 일반적인 의미의 데이터 소스에 '저장'할 수 없음<br/>
각 파티션의 내용을 저장하려면 전체 파티션을 순회하면서 외부 데이터베이스에 저장해야 함<br/>
이 방식은 고수준 API의 내부 처리 과정을 저수준 API로 구현하는 접근법임<br/>
스파크는 각 파티션의 데이터를 파일로 저장함<br/>

## 12.7.1 saveAsTextFile

텍스트 파일로 저장하려면 경로를 지정해야 함<br/>
필요한 경우 압축 코덱을 설정할 수도 있음<br/>

In [38]:
/*
words.saveAsTextFile("file:/tmp/bookTitle")
*/

압축 코덱을 설정하려면 하둡에서 사용 가능한 코덱을 임포트해야 함<br/>
org.apache.hadoop.io.compress 라이브러리에서 지원하는 코덱을 찾을 수 있음<br/>

In [39]:
/*
import org.apache.hadoop.io.compress.BZip2Codec

words.saveAsTextFile("file:/tmp/bookTitleCompressed", classOf[BZip2Codec])
*/

## 12.7.2 시퀀스 파일

스파크는 하둡 에코시스템을 기반으로 성장했으므로 다양한 하둡 기능과 잘 호환됨<br/>
시퀀스 파일은 바이너리 키-값 쌍으로 구성된 플랫 파일이며, 맵리듀스의 입출력 포맷으로 널리 사용됨<br/>

스파크는 saveAsObjectFile 메서드나 명시적인 키-값 쌍 데이터 저장 방식을 이용해 시퀀스 파일을 작성할 수 있음<br/>

In [40]:
/*
words.saveAsObjectFile("/tmp/my/sequenceFilePath")
*/

# 12.8 캐싱

RDD 캐싱에도 DataFrame이나 Dataset의 캐싱과 동일한 원칙이 적용됨<br/>
RDD를 캐시하거나 저장(persist)할 수 있음<br/>
기본적으로 캐시와 저장은 메모리에 있는 데이터만을 대상으로 함<br/>
setName 함수를 사용하면 캐시된 RDD에 이름을 지정할 수 있음<br/>

In [41]:
words.cache()

res35: words.type = myWords ParallelCollectionRDD[19] at parallelize at <console>:25


저장소 수준은 싱글턴 객체인 org.apache.spark.storage.StorageLevel의 속성(메모리, 디스크 또는 둘의 조합 그리고 off-heap 등이 있음) 중 하나로 지정 가능<br/>
저장소 수준을 지정하고 나면 다음 예제와 같이 저장소 수준을 조회할 수 있음<br/>
저장소 수준은 20장에서 자세히 알아보겠음<br/>

In [42]:
words.getStorageLevel

res36: org.apache.spark.storage.StorageLevel = StorageLevel(memory, deserialized, 1 replicas)


# 12.9 체크포인팅

DataFrame API에서 사용할 수 없는 기능 중 하나는 체크포인팅(checkpointing) 개념임<br/>
체크포인팅은 RDD를 디스크에 저장하는 방식임<br/>
나중에 저장된 RDD를 참조할 때는 원본 데이터 소스를 다시 계산해 RDD를 생성하지 않고 디스크에 저장된 중간 결과 파티션을 참조함<br/>
이런 동작은 메모리에 저장하지 않고 디스크에 저장한다는 사실만 제외하면 캐싱과 유사함<br/>
이 기능은 반복적인 연산 수행 시 매우 유용함<br/>

In [43]:
/*
spark.sparkContext.setCheckpointDir("/some/path/for/checkpointing")
words.checkpoint()
*/

이제 words RDD를 참조하면 데이터 소스의 데이터 대신 체크포인트에 저장된 RDD를 사용함<br/>
이것은 유용한 최적화 기법임<br/>

# 12.10 RDD를 시스템 명령으로 전송하기

pipe 메서드를 사용하면 파이핑 요소로 생성된 RDD를 외부 프로세스로 전달할 수 있음<br/>
이때 외부 프로세스는 파티션마다 한 번씩 처리해 결과 RDD를 생성함<br/>
각 입력 파티션의 모든 요소는 개행 문자 단위로 분할되어 여러 줄의 입력 데이터로 변경된 후 프로세스의 표준 입력(stdin)에 전달됨<br/>
결과 파티션은 프로세스의 표준 출력(stdout)으로 생성됨<br/>
이때 표준 출력의 각 줄은 출력 파티션의 하나의 요소가 됨<br/>
비어 있는 파티션을 처리할 때도 프로세스는 실행됨<br/>

사용자가 정의한 두 함수를 인수로 전달하면 출력 방식을 원하는 대로 변경할 수 있음<br/>

다음 예제는 각 파티션을 wc 명령에 연결할 수 있음<br/>
각 로우는 신규 로우로 전달되므로 로우 수를 세면 각 파티션별 로우 수를 얻을 수 있음<br/>

In [44]:
words.pipe("wc -l").collect()

res38: Array[String] = Array(5, 5)


위 예제에서는 파티션당 5개의 로우를 얻게 됨

## 12.10.1 mapPartitions

*이전 예제에서 스파크는 실제 코드를 실행할 때 파티션 단위로 동작한다는 사실을 알 수 있었음<br/>
또한 map 함수에서 반환하는 RDD의 진짜 형태가 MapPartitionsRDD라는 사실을 이미 알아차렸을 수도 있음*<br/>
**map은 mapPartitions의 로우 단위 처리를 위한 별칭일 뿐임<br/>
mapPartitions는 개별 파티션(이터레이터로 표현)에 대해 map 연산을 수행할 수 있음<br/>
그 이유는 클러스터에서 물리적인 단위로 개별 파티션을 처리하기 때문(로우 단위로 처리하지 않음)**<br/>
다음은 데이터의 모든 파티션에 '1' 값을 생성하고 표현식에 따라 파티션 수를 세어 합산함<br/>

In [45]:
words.mapPartitions(part => Iterator[Int](1)).sum() // 합계는 2

res39: Double = 2.0


이 메서드는 기본적으로 파티션 단위로 작업을 수행함<br/>
따라서 **전체** 파티션에 대한 연산을 수행할 수 있음<br/>
RDD의 전체 하위 데이터셋에 원하는 연산을 수행할 수 있으므로 아주 유용한 기능임<br/>
파티션 그룹의 전체 값을 단일 파티션으로 모든 다음 임의의 함수를 적용하고 제어할 수 있음<br/>
한 가지 예로 사용자가 정의한 머신러닝 알고리즘으로 파티션을 연결하고 일부 기업 데이터를 이용해 개별 모델을 학습하는 상황을 들 수 있음<br/>

mapPartitionsWithIndex 같이 mapPartitions와 유사한 기능을 제공하는 함수가 있음<br/>
mapPartitionsWithIndex 함수를 사용하려면 인덱스(파티션 범위의 인덱스)와 파티션의 모든 아이템을 순회하는 이터레이터를 가진 함수를 인수로 지정해야 함<br/>
파티션 인덱스는 RDD의 파티션 번호임<br/>
파티션 인덱스를 사용해 각 레코드가 속한 데이터셋이 어디에 있는지 알아낼 수 있음<br/>
그리고 이 정보를 디버깅에 활용할 수 있음<br/>
또한 이 기능을 이용해 map 함수가 정상적으로 동작하는지 시험해볼 수 있음<br/>

In [46]:
def indexedFunc(partitionIndex:Int, withinPartIterator:Iterator[String]) = {
    withinPartIterator.toList.map(
        value => s"Partition: $partitionIndex => $value").iterator
}

words.mapPartitionsWithIndex(indexedFunc).collect()

indexedFunc: (partitionIndex: Int, withinPartIterator: Iterator[String])Iterator[String]
res40: Array[String] = Array(Partition: 0 => Spark, Partition: 0 => The, Partition: 0 => Definitive, Partition: 0 => Guide, Partition: 0 => :, Partition: 1 => Big, Partition: 1 => Data, Partition: 1 => Processing, Partition: 1 => Made, Partition: 1 => Simple)


## 12.10.2 foreachPartition

mapPartitions 함수는 처리 결과를 반환하지만 foreachPartition 함수는 파티션의 모든 데이터를 순회할 뿐 결과는 반환하지 않음<br/>
즉, 반환값의 존재 여부가 두 함수의 차이점임<br/>
foreachPartition은 각 파티션의 데이터를 데이터베이스에 저장하는 것과 같이 개별 파티션에서 특정 작업을 수행하는 데 매우 적합한 함수임<br/>
실제로 많은 데이터 소스 커넥터에서 이 함수를 사용하고 있음<br/>
다음은 임의로 생성한 ID를 이용해 임시 디렉터리에 결과를 저장하는 텍스트 파일 소스를 자체 구현한 예제임<br/>

In [47]:
/*
words.foreachPartition { iter =>

    import java.io._
    import scala.util.Random

    val randomFileName = new Random().nextInt()
    val pw = new PrintWriter(new File(s"/tmp/random-file-${randomFileName}.txt"))
    
    while(iter.hasNext) {
        pw.write(iter.next())
    }
    pw.close()
}
*/

/tmp 디렉터리를 조회하면 2개의 파일을 찾을 수 있음<br/>

## 12.10.3 glom

glom 함수는 데이터셋의 모든 파티션을 배열로 변환하는 흥미로운 함수임<br/>
이 기능은 데이터를 드라이버로 모으고 데이터가 존재하는 파티션의 배열이 필요한 경우에 매우 유용함<br/>
하지만 파티션이 크거나 파티션 수가 많다면 드라이버가 비정상적으로 종료될 수 있으므로 심각한 안정성 문제를 일으킬 수도 있음<br/>

다음은 입력된 단어를 두 개의 파티션에 개별적으로 할당하는 예제임<br/>

In [48]:
spark.sparkContext.parallelize(Seq("Hello", "World"), 2).glom().collect() // Array(Array(Hello), Array(World))

res42: Array[Array[String]] = Array(Array(Hello), Array(World))


# 12.11 정리

이 장에서는 단일 RDD를 처리하는 방법과 RDD API의 기초를 알아보았음<br/>
다음 장에서는 조인과 키-값 RDD 같은 RDD 고급 개념을 알아보겠음<br/>